In [5]:
import os
import pandas as pd
from Bio import SeqIO
from Bio import Seq

os.getcwd()
os.chdir('/Volumes/AHN/captive_ape_microbiome')


In [6]:
%%bash 
cp results/moeller_sup/Bacteroidaceae_codiv.fna results/gyrb_all/Bt/moeller_sup/
cd results/gyrb_all/Bt

#make blastdb
makeblastdb -in moeller_sup/Bacteroidaceae_codiv.fna -dbtype nucl

#blast moeller co-div seqs
blastn -query ASVs_filtered.fasta -db moeller_sup/Bacteroidaceae_codiv.fna \
-outfmt "7 qseqid salltitles sseqid pident length qlen evalue" \
-out moeller_sup/ASVs_filtered_blastout.txt \
-max_target_seqs 5



Building a new DB, current time: 06/17/2020 15:47:04
New DB name:   /Volumes/AHN/captive_ape_microbiome/results/gyrb_all/Bt/moeller_sup/Bacteroidaceae_codiv.fna
New DB title:  moeller_sup/Bacteroidaceae_codiv.fna
Sequence type: Nucleotide
Deleted existing Nucleotide BLAST database named /Volumes/AHN/captive_ape_microbiome/results/gyrb_all/Bt/moeller_sup/Bacteroidaceae_codiv.fna
Keep MBits: T
Maximum file size: 1000000000B
Adding sequences from FASTA; added 208 sequences in 0.00553393 seconds.


In [15]:
#filter output to find 
blast_res = pd.read_csv('results/gyrb_all/Bt/moeller_sup/ASVs_filtered_blastout.txt',sep='\t',comment='#',header=None)
blast_res.columns = ['qseqid','salltitles','sseqid','pident','length','qlen','evalue']
blast_res.head()
blast_res = blast_res.groupby('qseqid').head(1).reset_index(drop=True)
blast_res_pass = blast_res[blast_res['pident']>95].reset_index(drop=True)
codiv_ASVs = list(blast_res_pass['qseqid'])


In [16]:
#find captive asvs
metadata = pd.read_csv('metadata/metadata_Bt_samples_gyrb.txt',sep='\t',index_col=None)
tax_table = pd.read_csv('results/gyrb_all/Bt/assigned_taxonomy/ASVs_filtered_taxonomy.txt',sep='\t',index_col=0)
asv_table = pd.read_csv('results/gyrb_all/Bt/ASVs_filtered_counts.tsv',sep='\t',index_col=0)
groups = ['wild_gorilla', 'wild_bonobo', 'wild_chimp']

def get_description(sample_name):
    description = metadata[metadata.index==sample_name]['Description'].to_string(index=False).replace(' ','')
    return(description)
def get_HR_clade(descriptions,numSamples):
    if len(descriptions) == 0:
        return('NA')
    elif len(descriptions) == 1:
        return(descriptions[0])
    else:
        return('notHR')

wildDict = {}
for index,row in asv_table.iterrows():
    samples = list(asv_table.columns[row>0])
    numSamples = len(samples)
    descriptions = list(set([get_description(x) for x in samples]))
    descriptions_filt = [x for x in descriptions if x in groups]   
    if len(descriptions_filt) > 0:
        wildDict[index] = descriptions_filt
wildASVs = list(wildDict.keys())

In [17]:
print(len(wildASVs))

905


In [18]:
wild_codiv_ASVs = list(set(wildASVs).intersection(set(codiv_ASVs)))
blast_res_pass[blast_res_pass['qseqid'].isin(wild_codiv_ASVs)]

#for index,row in asv_table.iterrows():
#    if index in captive_codiv_ASVs:
#        samples = list(asv_table.columns[row>0])
#        print(samples)
#        numSamples = len(samples)
 #       print(numSamples)
#        descriptions = list(set([get_description(x) for x in samples]))
#        print(descriptions)
#        descriptions_filt = [x for x in descriptions if x in groups]   

,qseqid,salltitles,sseqid,pident,length,qlen,evalue
0,ASV_2,Chimp9931019 Bt2_clade1_chimp,Chimp9931019,100.000,250,250,6.220000e-133
1,ASV_8,Human7859459 Bt3_clade1_human,Human7859459,99.600,250,250,2.890000e-131
2,ASV_18,Chimp9859517 Bt3_clade1_chimp,Chimp9859517,100.000,250,250,6.220000e-133
3,ASV_22,Chimp9228565 Bt2_clade2_chimp,Chimp9228565,100.000,250,250,6.220000e-133
4,ASV_25,Chimp9931019 Bt2_clade1_chimp,Chimp9931019,99.200,250,250,1.350000e-129
5,ASV_28,Chimp9931019 Bt2_clade1_chimp,Chimp9931019,99.600,250,250,2.890000e-131
6,ASV_39,Human6700665 Bt3_clade1_human,Human6700665,100.000,250,250,6.220000e-133
7,ASV_42,Chimp9931019 Bt2_clade1_chimp,Chimp9931019,99.600,250,250,2.890000e-131
9,ASV_44,Chimp5888147 Bt1_clade1_chimp,Chimp5888147,100.000,250,250,6.220000e-133
10,ASV_46,Chimp9859517 Bt3_clade1_chimp,Chimp9859517,99.600,250,250,2.890000e-131
